## Import library and dataset

In [1]:
import os
# os.environ["TOKENIZERS_PARALLELISM"] = "false"

from sklearn.metrics import accuracy_score, recall_score, f1_score
from sklearn.metrics import precision_score
from datasets import Dataset
from transformers import TrainingArguments, Trainer, GPT2Tokenizer, GPT2ForSequenceClassification,\
    EarlyStoppingCallback
import pandas as pd

dataset = pd.read_csv(f"../dataset/preprocess/github-labels-top3-803k-100.0%.csv")
print(dataset.issue_label.value_counts())
print(dataset.info())

issue_label
0.0    4014
1.0    3326
2.0     695
Name: count, dtype: int64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8035 entries, 0 to 8034
Data columns (total 11 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Unnamed: 0.2              8035 non-null   int64  
 1   Unnamed: 0.1              8035 non-null   int64  
 2   Unnamed: 0                8035 non-null   int64  
 3   issue_url                 8035 non-null   object 
 4   issue_label               8035 non-null   float64
 5   issue_created_at          8035 non-null   object 
 6   issue_author_association  8035 non-null   object 
 7   repository_url            8035 non-null   object 
 8   issue_title               8035 non-null   object 
 9   issue_body                7242 non-null   object 
 10  text                      8035 non-null   object 
dtypes: float64(1), int64(3), object(7)
memory usage: 690.6+ KB
None


## preprocess the dataset
### process the data

In [2]:
dataset = dataset.rename(columns={"issue_label": "label"})
# label required to be int64, otherwise will get error during training
dataset['label'] = dataset['label'].astype('int64')

pd.set_option('future.no_silent_downcasting', True)

### tokenize the data

In [3]:
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token

encodings = tokenizer(
    dataset.text.values.tolist(),
    max_length = 128,
    truncation = True,
    padding="max_length",
    return_attention_mask=True,
    return_tensors='pt'
)

print(encodings)

# dataset = Dataset.from_pandas(df)
# print(dataset)
# def preprocess(batch):
#     return tokenizer(batch["text"], padding="max_length", truncation=True, max_length=128)
#
# dataset = dataset.map(preprocess, batched=True)
# dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])

{'input_ids': tensor([[38816,  2581,    25,  ..., 50256, 50256, 50256],
        [  464,  9104,  1276,  ..., 50256, 50256, 50256],
        [   33,  3219, 18257,  ..., 50256, 50256, 50256],
        ...,
        [41181,   263,  4049,  ...,     8, 29705,   238],
        [16775, 22025,  1496,  ..., 50256, 50256, 50256],
        [ 3673,  6079,   287,  ...,    13, 12429,  8600]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 1, 1, 1]])}


### split the data

In [4]:
trainer_dataset = Dataset.from_dict({
    **encodings,
    "label": dataset.label.values
})

trainer_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])

print(trainer_dataset)

# how to do stratify
split_data = trainer_dataset.train_test_split(test_size=0.15)
train_set = split_data['train']
test_set = split_data['test']

Dataset({
    features: ['input_ids', 'attention_mask', 'label'],
    num_rows: 8035
})


### run the training

In [5]:
model = GPT2ForSequenceClassification.from_pretrained("gpt2", num_labels=3)
model.config.pad_token_id = tokenizer.pad_token_id

# Define training args
training_args = TrainingArguments(
    output_dir="../dataset",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    eval_strategy="epoch",
    logging_strategy="steps",
    logging_steps=10,
    num_train_epochs=50,
    save_strategy="epoch",
    load_best_model_at_end=True,
    report_to="none",
    metric_for_best_model="accuracy",
    greater_is_better=True,
)

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = logits.argmax(axis=-1)

    return {
        "accuracy": accuracy_score(labels, predictions),
        "precision": precision_score(labels, predictions, average="weighted"),
        "recall": recall_score(labels, predictions, average="weighted"),
        "f1": f1_score(labels, predictions, average="weighted"),
    }

# Define trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_set,
    eval_dataset=test_set,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
)

# Train the model
trainer.train()

# save model
# trainer.save_model("trained_electra_model")

# save tokenizer
# tokenizer.save_pretrained("")


Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/torch/utils/data/dataloader.py:684: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Epoch,Training Loss,Validation Loss


KeyboardInterrupt: 